In [ ]:
import gc
import glob
import json
import random
from collections import defaultdict
from pathlib import Path
from tempfile import gettempdir

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import scipy as sp
import scipy.constants
from IPython.core.display import HTML, display
from jupyter_dash import JupyterDash
from numpy import ma

from common import *

In [ ]:
# Setup plotting
JupyterDash.infer_jupyter_proxy_config()
COLORS = plotly.colors.DEFAULT_PLOTLY_COLORS

In [ ]:
# Find the default output directory
OUTPUT_DIR = Path(gettempdir()) / "boltzmann_solver" / "full"
if not OUTPUT_DIR.is_dir():
    OUTPUT_DIR = Path("/media/ssh/uni-josh/tmp/josh/boltzmann_solver/full")
if not OUTPUT_DIR.is_dir():
    OUTPUT_DIR = Path("/media/ssh/bakerlab11/tmp/jellis/boltzmann_solver/full")
print(f"Loading data from {OUTPUT_DIR}")

# Decay Only

## 1 Generation

In [ ]:
data = read_csv(OUTPUT_DIR / "decay_1.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "N1"])

In [ ]:
plot_density(data, ["H", "L1", "N1"])

## 3 Generations

In [ ]:
data = read_csv(OUTPUT_DIR / "decay_3.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

In [ ]:
plot_density(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

# Washout Only

## 1 Generation

In [ ]:
data = read_csv(OUTPUT_DIR / "washout_1.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "N1"])

In [ ]:
plot_density(data, ["H", "L1", "N1"])

## 3 Generation

In [ ]:
data = read_csv(OUTPUT_DIR / "washout_3.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

In [ ]:
plot_density(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

# Full

## 1 Generation

In [ ]:
data = read_csv(OUTPUT_DIR / "full_1.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "N1"])

In [ ]:
plot_density(data, ["H", "L1", "N1"])

## 3 Generation

In [ ]:
data = read_csv(OUTPUT_DIR / "full_3.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

In [ ]:
plot_density(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

# Miscellaneous

## Evolution

In [ ]:
data, ptcls = read_evolution(OUTPUT_DIR / "evolution.json")

In [ ]:
go.Figure(
    data=[go.Scatter(name=p, x=data["beta"], y=data[p, "mass"]) for p in ptcls],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Mass [GeV]", type="log", exponentformat="power"),
    ),
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(name=p, x=data["beta"], y=data[p, "mass"] * data["beta"])
        for p in ptcls
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(
            title="Mass / Temperatre", type="log", exponentformat="power"
        ),
    ),
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(name=p, x=data["beta"], y=data[p, "width"] / data[p, "mass"])
        for p in ptcls
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Width / Mass", type="log", exponentformat="power"),
    ),
)

## Higgs Equilibrium

In [ ]:
datas = list(
    map(read_csv, sorted(glob.glob(str(OUTPUT_DIR / "higgs_equilibrium" / "*.csv"))))
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(
            x=data["n"]["beta"],
            y=data["n"]["H"],
            mode="lines",
            line=go.scatter.Line(color=cmap("viridis", i / len(datas))),
            showlegend=False,
        )
        for i, data in enumerate(datas)
    ]
    + [
        go.Scatter(
            x=data["n"]["beta"],
            y=data["n"]["(H)"],
            mode="lines",
            line=go.scatter.Line(color="black"),
            showlegend=False,
        )
        for i, data in enumerate(datas)
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Width / Mass", type="log", exponentformat="power"),
    ),
)

## Lepton Equilibrium

In [ ]:
datas = list(
    map(read_csv, sorted(glob.glob(str(OUTPUT_DIR / "lepton_equilibrium" / "*.csv"))))
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(
            x=data["n"]["beta"],
            y=data["n"]["L1"],
            mode="lines",
            line=go.scatter.Line(color=cmap("viridis", i / len(datas))),
            showlegend=False,
        )
        for i, data in enumerate(datas)
    ]
    + [
        go.Scatter(
            x=data["n"]["beta"],
            y=data["n"]["(L1)"],
            mode="lines",
            line=go.scatter.Line(color="black"),
            showlegend=False,
        )
        for i, data in enumerate(datas)
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Width / Mass", type="log", exponentformat="power"),
    ),
)

## Gammas

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "gamma.csv")
data.drop(
    columns=list(filter(lambda x: "3" in x or "2" in x, data.columns)),
    inplace=True,
)
# print(data.columns)
plot_gamma(data)

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "asymmetry.csv")
plot_gamma(data)